# 求Westinghouse汽轮机高压缸相对内效率等参数

### 题目大意
国产引进型Westinghouse 300MW汽轮机，额定工况下主汽门前的水蒸气压力为16.7MPa，温度为537°C，主蒸汽流量为910.2t/h，给水温度为274°C，采用高中压合缸，一个双分流对置低压缸结构布置，各级主要设计数据已给出，求高压缸各级的理想焓降和实际焓降以及相对内效率等参数。

In [2]:
# -*- coding: utf-8 -*- 
from iapws import IAPWS97
from prettytable import PrettyTable

#定义存储额定工况数据的字典
ratedData = {"级号":["调节级","1","2","3","4","5","6","7","8","9","10","11"],
             "级流量":[905.6,886.1,886.1,886.1,886.1,886.1,886.1,886.1,852.9,852.9,852.9,852.9],
             "级前温度":[537.0,483.8,471.1,457.9,443.8,428.9,413.5,397.6,381.0,365.2,348.8,331.8],
             "级前压力":[16.00,11.60,10.70,9.87,9.01,8.17,7.38,6.61,5.89,5.26,4.66,4.10],
             "全级压差":[4.250,0.883,0.859,0.857,0.838,0.797,0.765,0.730,0.626,0.599,0.563,0.525],
             "内功率":[18555,5365,5608,5992,6325,6551,6855,7194,6345,6653,6930,7203]}

#定义存储结果的字典
result = {}

#计算实际焓降
actHd=[]
for i in range(12):
    r = ratedData["内功率"][i]/(ratedData["级流量"][i]*1000/3600)
    actHd.append(round(r,4))
result["实际焓降"]=actHd

#计算级前焓和级前比熵；使用Pipy提供的IAPWSIF97软件包计算
preH=[]
preS=[]
for i in range(12):
    r = IAPWS97(P=ratedData["级前压力"][i],T=(ratedData["级前温度"][i]+273.15))
    preH.append(round(r.h,4))
    preS.append(round(r.s,4))
result["级前焓"]=preH
result["级前比熵"]=preS

#计算级后焓,根据刚刚计算出的级前比熵和级后压力计算
#首先计算级后压力,然后使用Pipy提供的IAPWSIF97软件包计算级后焓
aftH=[]
for i in range(12):
    afterP = ratedData["级前压力"][i] - ratedData["全级压差"][i]
    r = IAPWS97(P=afterP,s=result["级前比熵"][i])
    aftH.append(float(str(round(r.h,4))))
result["级后焓"]=aftH

#计算理想焓降
dreHd=[]
for i in range(12):
    r = result["级前焓"][i] - result["级后焓"][i]
    dreHd.append(round(r,4))
result["理想焓降"]=dreHd

#计算相对内效率
eff=[]
for i in range(12):
    r = round(result["实际焓降"][i]/result["理想焓降"][i],4)
    eff.append(r)
result["相对内效率"]=eff

#以表格的形式呈现计算结果
pTable=PrettyTable(["级号","级前焓","级前比熵","级后焓","实际焓降","理想焓降","相对内效率"])
for i in range(12):
    pTable.add_row([ratedData["级号"][i],result["级前焓"][i],result["级前比熵"][i],result["级后焓"][i],
                    result["实际焓降"][i],result["理想焓降"][i],result["相对内效率"][i]])
print(pTable)

+--------+-----------+----------+-----------+----------+----------+------------+
|  级号  |   级前焓  | 级前比熵 |   级后焓  | 实际焓降 | 理想焓降 | 相对内效率 |
+--------+-----------+----------+-----------+----------+----------+------------+
| 调节级 | 3403.7387 |  6.439   | 3304.1824 |  73.761  | 99.5563  |   0.7409   |
|   1    |  3311.268 |  6.4537  |  3286.697 | 21.7966  |  24.571  |   0.8871   |
|   2    | 3289.3214 |  6.4579  | 3263.7551 | 22.7839  | 25.5663  |   0.8912   |
|   3    | 3265.8128 |  6.4595  | 3238.5406 |  24.344  | 27.2722  |   0.8926   |
|   4    | 3241.2056 |  6.4634  | 3212.4627 | 25.6969  | 28.7429  |   0.894    |
|   5    | 3215.1429 |  6.4673  | 3185.4712 | 26.6151  | 29.6717  |   0.897    |
|   6    | 3187.9818 |  6.4706  | 3157.0568 | 27.8501  |  30.925  |   0.9006   |
|   7    | 3160.2984 |  6.4758  | 3127.9628 | 29.2274  | 32.3356  |   0.9039   |
|   8    |  3131.056 |  6.4798  | 3100.6646 | 26.7816  | 30.3914  |   0.8812   |
|   9    | 3103.2761 |  6.4842  |  3071.347 | 28.0816  |

### 实验感受
* 实验前学习了字典和列表的用法，在实验过程中多处使用了字典和列表，感觉十分方便。
* 实验中只需要几个循环就可以快速得到想要的数据，减去了之前要用笔算和查软件的弊端。

### 实验中遇到的问题
* 实验中遇到了round函数截取特定位数的浮点数后，调用append加入list中时位数增多。如下一段代码所示

In [6]:
# -*- coding: utf-8 -*- 
from iapws import IAPWS97

ratedData = {"级号":["调节级","1","2","3","4","5","6","7","8","9","10","11"],
             "级流量":[905.6,886.1,886.1,886.1,886.1,886.1,886.1,886.1,852.9,852.9,852.9,852.9],
             "级前温度":[537.0,483.8,471.1,457.9,443.8,428.9,413.5,397.6,381.0,365.2,348.8,331.8],
             "级前压力":[16.00,11.60,10.70,9.87,9.01,8.17,7.38,6.61,5.89,5.26,4.66,4.10],
             "全级压差":[4.250,0.883,0.859,0.857,0.838,0.797,0.765,0.730,0.626,0.599,0.563,0.525],
             "内功率":[18555,5365,5608,5992,6325,6551,6855,7194,6345,6653,6930,7203]}

aftH=[]
aftH2=[]
for i in range(12):
    afterP = ratedData["级前压力"][i] - ratedData["全级压差"][i]
    r = IAPWS97(P=afterP,s=result["级前比熵"][i])
    print(round(r.h,4))
    aftH.append((round(r.h,4)))
    aftH2.append(float(str(round(r.h,4))))
    
#有问题的
print("有问题：",aftH)
#没有问题的
print("没有问题：",aftH2)

3304.1824
3286.697
3263.7551
3238.5406
3212.4627
3185.4712
3157.0568
3127.9628
3100.6646
3071.347
3041.3038
3010.1593
有问题： [3304.1824000000001, 3286.6970000000001, 3263.7550999999999, 3238.5405999999998, 3212.4627, 3185.4712, 3157.0567999999998, 3127.9627999999998, 3100.6646000000001, 3071.3470000000002, 3041.3038000000001, 3010.1592999999998]
没有问题： [3304.1824, 3286.697, 3263.7551, 3238.5406, 3212.4627, 3185.4712, 3157.0568, 3127.9628, 3100.6646, 3071.347, 3041.3038, 3010.1593]


* 从运行结果中可以看到，已经通过round函数截取了，打印出来也是4位小数，但是append到list中后就出现了问题，所以目前只能使用一个比较傻的方法，先转成string，再转成float。
* 不是所有的round函数都会出现这样的情况。
* 虽然问题解决了，但是问题的原因还是不明白。